In [1]:
import pandas as pd
from datetime import datetime
from collections import deque
from decimal import Decimal
import random

from strat_backtest.base.gen_trades import GenTrades, TradingConfig, RiskConfig

In [2]:
df = pd.read_parquet("../data/AAPL.parquet")
df

,Ticker,Open,High,Low,Close,Volume
Date,,,,,,
2020-01-02,AAPL,71.72,72.78,71.47,72.72,135480400.0
2020-01-03,AAPL,71.94,72.77,71.78,72.01,146322800.0
2020-01-06,AAPL,71.13,72.62,70.88,72.58,118387200.0
2020-01-07,AAPL,72.59,72.85,72.02,72.24,108872000.0
2020-01-08,AAPL,71.94,73.71,71.94,73.40,132079200.0
...,...,...,...,...,...,...
2025-05-09,AAPL,198.74,200.28,197.28,198.27,36453900.0
2025-05-12,AAPL,210.97,211.27,206.75,210.79,63775800.0
2025-05-13,AAPL,210.43,213.40,209.00,212.93,51909300.0


In [3]:
df1 = df.loc["2025-04-04":, :].head(10).reset_index()
df1

,Date,Ticker,Open,High,Low,Close,Volume
0,2025-04-04,AAPL,193.64,199.62,187.09,188.13,125910900.0
1,2025-04-07,AAPL,176.97,193.90,174.39,181.22,160466300.0
2,2025-04-08,AAPL,186.46,190.09,168.99,172.19,120859500.0
3,2025-04-09,AAPL,171.72,200.35,171.66,198.59,184395900.0
4,2025-04-10,AAPL,188.82,194.52,182.76,190.17,121880000.0
5,2025-04-11,AAPL,185.86,199.28,185.82,197.89,87435900.0
6,2025-04-14,AAPL,211.16,212.66,200.90,202.25,101352900.0
7,2025-04-15,AAPL,201.60,203.24,199.54,201.88,51343900.0
8,2025-04-16,AAPL,198.10,200.44,192.12,194.02,59732400.0
9,2025-04-17,AAPL,196.94,198.57,194.17,196.72,51334300.0


In [6]:
df1.columns = [col.lower() for col in df1.columns]
df1

,date,ticker,open,high,low,close,volume
0,2025-04-04,AAPL,193.64,199.62,187.09,188.13,125910900.0
1,2025-04-07,AAPL,176.97,193.90,174.39,181.22,160466300.0
2,2025-04-08,AAPL,186.46,190.09,168.99,172.19,120859500.0
3,2025-04-09,AAPL,171.72,200.35,171.66,198.59,184395900.0
4,2025-04-10,AAPL,188.82,194.52,182.76,190.17,121880000.0
5,2025-04-11,AAPL,185.86,199.28,185.82,197.89,87435900.0
6,2025-04-14,AAPL,211.16,212.66,200.90,202.25,101352900.0
7,2025-04-15,AAPL,201.60,203.24,199.54,201.88,51343900.0
8,2025-04-16,AAPL,198.10,200.44,192.12,194.02,59732400.0
9,2025-04-17,AAPL,196.94,198.57,194.17,196.72,51334300.0


In [7]:
df1["entry_signal"] = [
    "wait",
    "buy",
    "buy",
    "wait",
    "wait",
    "buy",
    "wait",
    "wait",
    "wait",
    "buy",
]
df1["exit_signal"] = [
    "sell",
    "wait",
    "wait",
    "wait",
    "sell",
    "wait",
    "wait",
    "sell",
    "wait",
    "wait",
]
df1

,date,ticker,open,high,low,close,volume,entry_signal,exit_signal
0,2025-04-04,AAPL,193.64,199.62,187.09,188.13,125910900.0,wait,sell
1,2025-04-07,AAPL,176.97,193.90,174.39,181.22,160466300.0,buy,wait
2,2025-04-08,AAPL,186.46,190.09,168.99,172.19,120859500.0,buy,wait
3,2025-04-09,AAPL,171.72,200.35,171.66,198.59,184395900.0,wait,wait
4,2025-04-10,AAPL,188.82,194.52,182.76,190.17,121880000.0,wait,sell
5,2025-04-11,AAPL,185.86,199.28,185.82,197.89,87435900.0,buy,wait
6,2025-04-14,AAPL,211.16,212.66,200.90,202.25,101352900.0,wait,wait
7,2025-04-15,AAPL,201.60,203.24,199.54,201.88,51343900.0,wait,sell
8,2025-04-16,AAPL,198.10,200.44,192.12,194.02,59732400.0,wait,wait
9,2025-04-17,AAPL,196.94,198.57,194.17,196.72,51334300.0,buy,wait


In [10]:
df1.loc[df1["entry_signal"] == df1["exit_signal"], :]

,date,ticker,open,high,low,close,volume,entry_signal,exit_signal
3,2025-04-09,AAPL,171.95,200.61,171.89,198.85,184395900,wait,wait
6,2025-04-14,AAPL,211.44,212.94,201.16,202.52,101352900,wait,wait
8,2025-04-16,AAPL,198.36,200.70,192.37,194.27,59732400,wait,wait


In [8]:
df1.to_parquet("../tests/data/sample_gen_trades.parquet", index=False)

temp = pd.read_parquet("../tests/data/sample_gen_trades.parquet")
temp

,date,ticker,open,high,low,close,volume,entry_signal,exit_signal
0,2025-04-04,AAPL,193.64,199.62,187.09,188.13,125910900.0,wait,sell
1,2025-04-07,AAPL,176.97,193.90,174.39,181.22,160466300.0,buy,wait
2,2025-04-08,AAPL,186.46,190.09,168.99,172.19,120859500.0,buy,wait
3,2025-04-09,AAPL,171.72,200.35,171.66,198.59,184395900.0,wait,wait
4,2025-04-10,AAPL,188.82,194.52,182.76,190.17,121880000.0,wait,sell
5,2025-04-11,AAPL,185.86,199.28,185.82,197.89,87435900.0,buy,wait
6,2025-04-14,AAPL,211.16,212.66,200.90,202.25,101352900.0,wait,wait
7,2025-04-15,AAPL,201.60,203.24,199.54,201.88,51343900.0,wait,sell
8,2025-04-16,AAPL,198.10,200.44,192.12,194.02,59732400.0,wait,wait
9,2025-04-17,AAPL,196.94,198.57,194.17,196.72,51334300.0,buy,wait


In [11]:
all(isinstance(dt, datetime) for dt in temp["date"])

True

In [4]:
temp = pd.read_parquet("../tests/data/sample_gen_trades.parquet")
temp

,date,ticker,open,high,low,close,volume,entry_signal,exit_signal
0,2025-04-04,AAPL,193.64,199.62,187.09,188.13,125910900.0,wait,sell
1,2025-04-07,AAPL,176.97,193.90,174.39,181.22,160466300.0,buy,wait
2,2025-04-08,AAPL,186.46,190.09,168.99,172.19,120859500.0,buy,wait
3,2025-04-09,AAPL,171.72,200.35,171.66,198.59,184395900.0,wait,wait
4,2025-04-10,AAPL,188.82,194.52,182.76,190.17,121880000.0,wait,sell
5,2025-04-11,AAPL,185.86,199.28,185.82,197.89,87435900.0,buy,wait
6,2025-04-14,AAPL,211.16,212.66,200.90,202.25,101352900.0,wait,wait
7,2025-04-15,AAPL,201.60,203.24,199.54,201.88,51343900.0,wait,sell
8,2025-04-16,AAPL,198.10,200.44,192.12,194.02,59732400.0,wait,wait
9,2025-04-17,AAPL,196.94,198.57,194.17,196.72,51334300.0,buy,wait


In [8]:
trades = pd.read_parquet("../sample_trades.parquet")
trades

,ticker,entry_datetime,entry_action,entry_lots,entry_price,exit_datetime,exit_action,exit_lots,exit_price,days_held,profit_loss,percent_ret,daily_ret,win
0,AAPL,2025-04-07,buy,10,181.22,2025-04-10,sell,10,190.17,3,8.95,0.049387,0.016199,1
1,AAPL,2025-04-08,buy,10,172.19,2025-04-15,sell,10,201.88,7,29.69,0.172426,0.022985,1
2,AAPL,2025-04-11,buy,10,197.89,2025-04-17,sell,10,196.72,6,-1.17,-0.005912,-0.000988,0
